In [6]:
import ast
import requests
import pandas as pd
import numpy as np
import sys
sys.path.append("../")
#scr
from scr.todas_zara import get_id_info
from scr.todas_zara import get_stores_zara

In [14]:
product= 7765500250

store = {
    "physicalStoreId":6479
}


stock = "https://itxrest.inditex.com/LOMOServiciosRESTCommerce-ws/common/1/stock/campaign/V2021/product/part-number/{}"
headers ={"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 11_2_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36"}
res = requests.get(stock.format(f"0{product}"), params=store, headers=headers).json()

In [15]:
def get_stock_zara(search_terms,lat,lng):
        
        products = get_id_info(search_terms)

        store_info = get_stores_zara(lat, lng)
        store_ids = [element["id"] for element in store_info]

        stock_url = "https://itxrest.inditex.com/LOMOServiciosRESTCommerce-ws/common/1/stock/campaign/V2021/product/part-number/{}"
        stock_params = {"physicalStoreId": store_ids}

        urls = [stock_url.format("".join(p["id_prod"]).rjust(11,"0")) for p in products]
        
        for i,product in enumerate(products):
            headers ={"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 11_2_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36"}
            products[i]["stock"] = requests.get(urls[i],params=stock_params,headers=headers).json()
        
        return products

In [16]:
prod = get_stock_zara("blusa morada",40.416050951604056, -3.7072654155715603)

In [17]:
prod

[{'url': 'https://www.zara.com/es/es/blusa-lino-tie-dye-limited-edition-p05598062.html',
  'image': 'https://static.zara.net/photos///2021/V/0/1/p/5598/062/611/2/w/260/5598062611_1_1_1.jpg?ts=1619601149775',
  'product_name': 'BLUSA LINO TIE DYE LIMITED EDITION',
  'price': '39,95 EUR',
  'id_prod': ['5598062', '611', ''],
  'stock': {'stocks': [{'datatype': 'stock',
     'physicalStoreId': 62,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 3,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 13, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 8, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 7, 'size': 104},
      {'datatype': 'sizeStock', 'sizeId': 5, 'quantity': 2, 'size': 105}]},
    {'datatype': 'stock',
     'physicalStoreId': 57,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 2,
       'size': 101},
      {'datatype': 'sizeStock',

Funtion to get `all` the info for the products

In [19]:
def products_final_zara(search_item, lat, long):
    
    for product_ in prod:
        print(f"Pinche aquí para comprar la prenda: ", product_["url"])
        print(f"Pinche aquí para ver la prenda: ", product_["image"])
        print(f"Producto: ", product_["product_name"])
        print(f"Precio: ", product_["price"])
    
        for stock in product_["stock"]["stocks"]:
       
            size_stk = []
        
            for sizeStock in stock["sizeStocks"]: 
            
                size_stock = {}
            
          
                size_ = {101:"XS", 102:"S", 103:"M", 104:"L", 105:"XL", 106:"XXL",125:"S-M",131:"L-XL"}
                if sizeStock["size"] in size_:
                    sizeStock["size"] = size_[sizeStock["size"]]
               
                    size_stock["Talla"] = sizeStock["size"]
                else:
                    size_stock["Talla"] = sizeStock["size"]
                
           
                if sizeStock["quantity"] <= 0:
                   size_stock["Stock"] = "Lo siento, no tenemos existencias!☹️"
                elif sizeStock["quantity"] == 1:
                    size_stock["Stock"] = "Últimas existencias!!"
                
                elif sizeStock["quantity"] >= 2:
                    size_stock["Stock"] = "El producto está en tienda!🙂"
                
                elif sizeStock["quantity"] >= 4:
                    size_stock["Stock"] = "Suficientes existencias!!😁"
                
                if size_stock["Stock"] == "No tenemos existencias! 🥲":
                    pass
                else: 
                    size_stk.append(size_stock)
            
        
        
            if len(size_stk) > 0:
                print("id tienda:",stock["physicalStoreId"])
                df = pd.DataFrame(size_stk)
                print(df)

In [20]:
products_final_zara("blusa morada", 40.416050951604056, -3.7072654155715603)

Pinche aquí para comprar la prenda:  https://www.zara.com/es/es/blusa-lino-tie-dye-limited-edition-p05598062.html
Pinche aquí para ver la prenda:  https://static.zara.net/photos///2021/V/0/1/p/5598/062/611/2/w/260/5598062611_1_1_1.jpg?ts=1619601149775
Producto:  BLUSA LINO TIE DYE LIMITED EDITION
Precio:  39,95 EUR
id tienda: 62
  Talla                         Stock
0    XS  El producto está en tienda!🙂
1     S  El producto está en tienda!🙂
2     M  El producto está en tienda!🙂
3     L  El producto está en tienda!🙂
4    XL  El producto está en tienda!🙂
id tienda: 57
  Talla                                 Stock
0    XS          El producto está en tienda!🙂
1     S          El producto está en tienda!🙂
2     M          El producto está en tienda!🙂
3     L  Lo siento, no tenemos existencias!☹️
4    XL  Lo siento, no tenemos existencias!☹️
id tienda: 198
  Talla                                 Stock
0     S                 Últimas existencias!!
1    XL  Lo siento, no tenemos existencias!☹